<a href="https://colab.research.google.com/github/Anspire/Notebooks/blob/master/GAN_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
from torch import nn, optim
from torch.autograd.variable import Variable

import torchvision
import torchvision.transforms as transforms

In [0]:
# Preprocessing
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5,), (0.5,))])

# Training data
train_set = torchvision.datasets.MNIST(root='.',
                                      train=True,
                                      download=True,
                                      transform=transform)
train_loader = torch.utils.data.DataLoader(train_set,
                                          batch_size=32,
                                          shuffle=True)

# Labels
classes = [str(i) for i in range(0,10)]
print(classes)

0it [00:00, ?it/s]

100%|█████████▉| 9895936/9912422 [00:34<00:00, 220629.76it/s]

Extracting ./MNIST/raw/train-images-idx3-ubyte.gz



0it [00:00, ?it/s]


 57%|█████▋    | 16384/28881 [00:00<00:00, 86224.52it/s]
32768it [00:00, 56551.03it/s]                           
0it [00:00, ?it/s]

Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz



 98%|█████████▊| 1613824/1648877 [00:05<00:00, 366293.84it/s]

0it [00:00, ?it/s]

Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz




  0%|          | 0/4542 [00:00<?, ?it/s]

8192it [00:00, 21184.36it/s]            

Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!
['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']


In [0]:
# Our Discriminator class
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(784, 1024),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(1024, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        out = self.model(x.view(x.size(0), 784))
        out = out.view(out.size(0), -1)
        return out.cuda()
        
discriminator = Discriminator()

In [0]:
# Our Generator class
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(100, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, 1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, 784),
            nn.Tanh()
        )

    def forward(self, x):
        x = x.view(x.size(0), 100)
        out = self.model(x).cuda()
        return out

generator = Generator()

In [0]:
# If we have a GPU with CUDA, use it
if torch.cuda.is_available():
    print("Using CUDA")
    discriminator.cuda()
    generator.cuda()

# Setup loss function and optimizers
lr = 0.0001
num_epochs = 40
num_batches = len(train_loader)

criterion = nn.BCELoss()
d_optimizer = torch.optim.Adam(discriminator.parameters(), lr=lr)
g_optimizer = torch.optim.Adam(generator.parameters(), lr=lr)

Using CUDA


In [0]:
# Convenience function for training our Discriminator
def train_discriminator(discriminator, real_images, real_labels, fake_images, fake_labels):
    discriminator.zero_grad()

    # Get the predictions, loss, and score of the real images
    predictions = discriminator(real_images)
    real_loss = criterion(predictions, real_labels)
    real_score = predictions

    # Get the predictions, loss, and score of the fake images
    predictions = discriminator(fake_images)
    fake_loss = criterion(predictions, fake_labels)
    fake_score = predictions

    # Calculate the total loss, update the weights, and update the optimizer
    d_loss = real_loss + fake_loss
    d_loss.backward()
    d_optimizer.step()
    return d_loss, real_score, fake_score

# Convenience function for training our Generator
def train_generator(generator, discriminator_outputs, real_labels):
    generator.zero_grad()

    # Calculate the total loss, update the weights, and update the optimizer
    g_loss = criterion(discriminator_outputs, real_labels)
    g_loss.backward()
    g_optimizer.step()
    return g_loss
    
for epoch in range(num_epochs):
    for n, (images, _) in enumerate(train_loader):

        # (1) Prepare the real data for the Discriminator
        real_images = Variable(images).cuda()
        real_labels = Variable(torch.ones(images.size(0))).cuda()

        # (2) Prepare the random noise data for the Generator
        noise = Variable(torch.randn(images.size(0), 100)).cuda()

        # (3) Prepare the fake data for the Discriminator
        fake_images = generator(noise)
        fake_labels = Variable(torch.zeros(images.size(0))).cuda()

        # (4) Train the discriminator on real and fake data
        d_loss, real_score, fake_score = train_discriminator(discriminator,
                                                             real_images, real_labels,
                                                             fake_images, fake_labels)

        # (5a) Generate some new fake images from the Generator.
        # (5b) Get the label predictions of the Discriminator on that fake data.
        noise = Variable(torch.randn(images.size(0), 100)).cuda()
        fake_images = generator(noise)

        outputs = discriminator(fake_images)

        # (6) Train the generator
        g_loss = train_generator(generator, outputs, real_labels)
    print(epoch)